<a href="https://colab.research.google.com/github/andlljr/gridworld/blob/main/GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explicação

Gerar rotas para um agente em um ambiente de gridworld de 8x8 em Python com pontos de início e chegada, bem como células de montanha, onde o agente não pode entrar, e células de areia movediça que capturam o agente e encerram o jogo.

Nesse exercício deixei que todas as informações sejam parametrizáveis, ou seja, o número de montanha, o número de areia movediça, ponto de inicio e chegada.

In [149]:
# Bibliotecas
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle

# Execução

In [150]:
def CriarGrid(grid_linha, grid_coluna):
    """
    Cria uma grid para um jogo ou visualização.

    Parâmetros:
    - grid_linha (int): O número de linhas da grade.
    - grid_coluna (int): O número de colunas da grade.

    Retorna:
    - board (list): Uma grade bidimensional representando o ambiente do jogo ou visualização,
                    onde cada posição é marcada com um número específico que representa seu tipo.
                    0 é espaço vazio.

    Exemplo de uso:
    >>> CriarGrid(5, 5)
    [[0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
    """

    return [[0 for _ in range(grid_coluna)] for _ in range(grid_linha)]

def DemarcarInicioFim(board, inicio, fim):
    """
    Demarca os valores de início e fim em uma grade.

    Parâmetros:
    - board (list): Uma grade bidimensional representando o ambiente do jogo ou visualização.
    - inicio (list): Lista contendo as coordenadas [linha, coluna] do início do jogador.
    - fim (list): Lista contendo as coordenadas [linha, coluna] do objetivo final.

    Retorna:
    - board (list): A grade bidimensional atualizada com os valores de início e fim demarcados.
                    10 é o início e 11 é o fim.

    Exemplo de uso:
    >>> board = CriarGrid(8, 8)
    >>> DemarcarInicioFim(board, [0, 0], [7, 7])
    [[10, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 11]]
    """

    inicio_linha, inicio_coluna = inicio
    fim_linha, fim_coluna = fim
    board[inicio_linha][inicio_coluna] = 10
    board[fim_linha][fim_coluna] = 11
    return board

In [151]:
def AreiaAleatoria(d_board, areia):
    """
    Cria posições aleatórias das areias para a grid.

    Parâmetros:
    - d_board (list[][]): Grid na qual irá inserir a areia.
    - areia (int): O número de areia.

    Retorna:
    - areia (list): Retorna uma lista com todas as posições de areia.

    Exemplo de uso:
    >>> CriarGrid([[10, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 11]],
     5
    )
    """
    areias = []
    for i in range(areia):
        linha_posicao = random.randint(0, len(d_board) - 1)
        coluna_posicao = random.randint(0, len(d_board[0]) - 1)  # Corrigido para pegar o número de colunas corretamente
        pos = [linha_posicao, coluna_posicao]
        if pos in areias or d_board[linha_posicao][coluna_posicao] == 10 or d_board[linha_posicao][coluna_posicao] == 11:
            i -= 1  # Corrigido para decrementar i corretamente
        else:
            areias.append(pos)
    return areias

def MontanhaAleatoria(d_board, areias, montanha):
    """
    Cria posições aleatórias das montanhas para a grid.

    Parâmetros:
    - d_board (list[][]): Grid na qual irá inserir a montanha.
    - montanha (int): O número de montanha.

    Retorna:
    - montanha (list): Retorna uma lista com todas as posições de montanha.

    Exemplo de uso:
    >>> CriarGrid([[10, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 11]],
     [[0,1], [0,2], [0,3], [3,4], [2,4]],
     2
    )
    """
    montanhas = []
    for i in range(montanha):
        linha_posicao = random.randint(0, len(d_board) - 1)
        coluna_posicao = random.randint(0, len(d_board[0]) - 1)  # Corrigido para pegar o número de colunas corretamente
        pos = [linha_posicao, coluna_posicao]
        if pos in montanhas or pos in areias or d_board[linha_posicao][coluna_posicao] == 10 or d_board[linha_posicao][coluna_posicao] == 11:
            i -= 1  # Corrigido para decrementar i corretamente
        else:
            montanhas.append(pos)
    return montanhas


def AdicionarAreiaseMontanhas(d_board, areias, montanhas):
    """
    Insere as areias e as montanhas no board, marcando o número 12 nas areias e o número 13 nas montanhas

    Parâmetros:
    - d_board (list[][]): Grid na qual irá inserir a montanha.
    - montanha (int): O número de montanha.

    Retorna:
    - montanha (list): Retorna uma lista com todas as posições de montanha.

    Exemplo de uso:
    >>> CriarGrid([[10, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 4],
     [0, 1, 2, 3, 11]],
     [[0,1], [0,2], [0,3], [3,4], [2,4]],
     [[2,3], [3,1]]
    )
    """

    for pos in areias:
        posx, posy = pos
        d_board[posx][posy] = 12

    for pos in montanhas:
        posx, posy = pos
        d_board[posx][posy] = 13

    return d_board


In [152]:
def DesenharGrid(board):
    """
    Desenha o grid com cores representando diferentes elementos.

    Parâmetros:
    - board (list[][]): Grid que será desenhado.

    Retorna:
    - Exibição do grid.

    Exemplo de uso:
    >>> DesenharGrid([[10, 1, 2, 3, 4],
                      [0, 1, 2, 3, 4],
                      [0, 1, 2, 3, 4],
                      [0, 1, 2, 3, 4],
                      [0, 1, 2, 3, 11]])
    """

    N = len(board)

    # Define colormap
    color_map = {13: [255, 0, 0],   # Red - Montanha
                 10: [0, 255, 0],   # Green - Início
                 11: [0, 0, 255],   # Blue - Fim
                 12: [255, 255, 0], # Yellow - Areia
                 0: [127, 255, 212] # Skyblue - Espaço vazio
                 }

    # Create a 3D numpy array that has a color channel dimension
    data_3d = np.zeros((N, N, 3), dtype=np.uint8)
    for i in range(N):
        for j in range(N):
            data_3d[i][j] = color_map[board[i][j]]

    # Display the plot
    plt.imshow(data_3d, interpolation='none', extent=[0, N, 0, N])

    # Add black borders to each cell
    for i in range(N):
        for j in range(N):
            rect = plt.Rectangle((j, i), 1, 1, linewidth=1, edgecolor='black', facecolor='none')
            plt.gca().add_patch(rect)

    plt.axis('off')
    plt.show()

In [153]:
def Ações(x, pos ,tamanho_grid , montanhas, politicas, areias, fim):
    linha, coluna = pos
    nova_linha = linha
    nova_coluna = coluna
    p = 0

    if x == 1:  # Cima
      nova_linha = linha - 1
      nova_coluna = coluna
      p = politicas[0]
    elif x == 2:  # Direita
      nova_linha = linha
      nova_coluna = coluna + 1
      p = politicas[1]
    elif x == 3:  # Baixo
      nova_linha = linha + 1
      nova_coluna = coluna
      p = politicas[2]
    elif x == 4:  # Esquerda
      nova_linha = linha
      nova_coluna = coluna - 1
      p = politicas[3]
    else:
      return f"Ação {x} não existe, sua ação deve ser [1,2,3,4]"
    # Verificar se a nova posição está dentro dos limites do tabuleiro e não é uma montanha
    if (nova_linha < 0 or nova_linha >= tamanho_grid[0]) or (nova_coluna < 0 or nova_coluna >= tamanho_grid[1]) or (Parada(pos, areias, fim)):
      return linha, coluna, p
    else:
      return nova_linha, nova_coluna, p

In [191]:
def Recompensas(posicao, fim, areias):
  if posicao == fim or posicao == [0,0]:
      return 0
  else:
      return -1

In [155]:
def Parada(pos, areias, fim):
  if pos in areias or pos in fim or pos == [0,0]:
    return True
  else:
    return False

In [156]:
def Recompensa_esperada(pos, desconto, areias, fim, montanhas, tamanho_grid, board, politica):
    valor = 0
    for i in range(1, 5):
        x, y, p = Ações(i, pos, tamanho_grid, montanhas, politica, areias, fim)
        recompensa = Recompensas(pos, fim, areias)
        valor += p * (recompensa + desconto * board[x][y])
    return valor

In [208]:
def Avaliar_politica(board, politica, areias, montanhas, fim, passos=1, desconto=1, in_place=False):
  for i in range(passos):
    novo_board = [linha[:] for linha in board]
    for j in range(len(board)):
      for k in range(len(board[j])):
        pos = [j, k]
        valor = Recompensa_esperada(pos, desconto, areias, fim, montanhas,[len(board), len(board[0])], board, politica)
        novo_board[j][k] = round(valor * desconto, 3)
    board = novo_board
  return board

# Teste

In [209]:
inicio = [0,0]
fim = [3,3]
politica = [0.25, 0.25, 0.25, 0.25]

board = CriarGrid(4, 4)
# Exemplo de uso:
novoBoard = [linha[:] for linha in board]  # Crie um novo tabuleiro para evitar alterar o original
areias = []
montanhas = []
novoBoard = AdicionarAreiaseMontanhas(novoBoard, areias, montanhas)

valor_tabela = Avaliar_politica(novoBoard, politica, areias, montanhas, fim, passos=10)

In [210]:
# Exemplo de uso:
inicio = [0, 0]
fim = [7, 7]
board = CriarGrid(8, 8)
areias = AreiaAleatoria(board, 5)
montanhas = MontanhaAleatoria(board, areias, 2)
board = AdicionarAreiaseMontanhas(board, areias, montanhas)
valor_tabela = Avaliar_politica(board, [0.25, 0.25, 0.25, 0.25], areias, montanhas, fim)

In [211]:
valor_tabela

[[12.0, 2.0, -1.0, -1.0, -1.0, -1.0, 2.0, 11.0],
 [2.0, -1.0, -1.0, -1.0, -1.0, -1.0, 2.0, 2.0],
 [-1.0, -1.0, -1.0, -1.0, -1.0, 2.0, 11.0, 2.0],
 [-1.0, -1.0, 2.0, -1.0, -1.0, -1.0, 2.0, -1.0],
 [-1.0, 2.0, 11.0, 2.0, -1.0, -1.0, 2.25, -1.0],
 [-1.0, 2.0, 2.0, -1.0, -1.0, 2.25, 2.25, 2.25],
 [2.0, 11.0, 2.0, -1.0, -1.0, 2.25, 2.25, 2.25],
 [-1.0, 2.0, -1.0, -1.0, -1.0, -1.0, 2.25, 0.0]]

In [212]:
# Exemplo de uso:
inicio = [0, 0]
fim = [7, 7]
board = CriarGrid(8, 8)
areias = AreiaAleatoria(board, 5)
montanhas = MontanhaAleatoria(board, areias, 2)
board = AdicionarAreiaseMontanhas(board, areias, montanhas)
valor_tabela = Avaliar_politica(board, [0.1, 0.4, 0.4, 0.1], areias, montanhas, fim)

In [213]:
valor_tabela

[[0.0, -1.0, -1.0, -1.0, 3.8, -1.0, -1.0, -1.0],
 [-1.0, 4.2, -1.0, 3.8, 11.0, 0.2, 3.8, -1.0],
 [4.2, -1.0, 0.3, 4.2, 0.2, 8.6, 11.0, 0.2],
 [-1.0, 0.3, 9.0, -1.0, 5.1, 11.0, 1.4, -1.0],
 [-1.0, 3.8, 11.0, 1.5, 3.8, 0.2, -1.0, -1.0],
 [-1.0, -1.0, 0.2, 3.8, 11.0, 0.2, -1.0, -1.0],
 [-1.0, -1.0, -1.0, -1.0, 0.2, -1.0, -1.0, -1.0],
 [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.0]]

In [190]:
print(areias)

[[1, 4], [2, 1], [3, 4], [4, 4], [7, 6]]
